In [3]:
!nvidia-smi

Fri May  3 12:09:55 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.40.06              Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   47C    P8             11W /  130W |    7080MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
!curl http://0.0.0.0:8000/v1/models

{"object":"list","data":[{"id":"solidrust/Meta-Llama-3-8B-Instruct-hf-AWQ","object":"model","created":1714734632,"owned_by":"vllm","root":"solidrust/Meta-Llama-3-8B-Instruct-hf-AWQ","parent":null,"permission":[{"id":"modelperm-d4b644a1e0bb4eeba97216c1e307b60c","object":"model_permission","created":1714734632,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [16]:
from openai import OpenAI
import concurrent.futures
import time

client = OpenAI(base_url="http://0.0.0.0:8000/v1", api_key="sk-xxx") # dummy key


def get_response(name, max_tokens = 100, prompt = "Write a story about llamas.", system_prompt = "You are a story writing assistant."):
    t0 = time.time()
    response = client.chat.completions.create(
        max_tokens=max_tokens,
        model="solidrust/Meta-Llama-3-8B-Instruct-hf-AWQ",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": prompt
            }
        ])
    return name, response, time.time() - t0


In [14]:
name, response, t = get_response(0, max_tokens=250)
print(f'Took {t}s')
print(response.choices[0].message.content)

Took 4.858847618103027s
In the small village of Pachamama, nestled in the Andean mountains, a group of llamas held a special place in the hearts of the villagers. These majestic creatures, with their soft fur and gentle eyes, had been a part of the community for generations.

The story begins with a young llama named Luna, who lived on a farm on the outskirts of the village. Luna was the most curious and adventurous of the llamas, always looking for new experiences and exploring the surrounding countryside. One day, while wandering through the nearby forest, Luna stumbled upon a hidden clearing deep in the woods.

As she entered the clearing, she noticed a small, intricately carved wooden door hidden behind a waterfall. The door was adorned with symbols and patterns, which Luna didn't recognize. But she felt an inexplicable connection to the door, as if it held a secret just for her.

Without hesitation, Luna pushed the door open and discovered a cozy, hidden room filled with books, sc

In [15]:


t0 = time.time()
threads = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit the tasks to the executor
    for i in range(10):
        threads.append(executor.submit(get_response, i, 250))
        print('Submited thread', i)

    # print results as and when they come in
    for task in concurrent.futures.as_completed(threads):
        name, response, exec_time = task.result()
        message = response.choices[0].message.content
        print(f'Request {name}, time: {exec_time:.2f}, n_tokens = {response.usage.total_tokens}')
        #print(message)

print('Total Execution time:', time.time()-t0)

Submited thread 0
Submited thread 1
Submited thread 2
Submited thread 3
Submited thread 4
Submited thread 5
Submited thread 6
Submited thread 7
Submited thread 8
Submited thread 9
Request 3, time: 6.48, n_tokens = 279
Request 6, time: 6.46, n_tokens = 279
Request 0, time: 6.50, n_tokens = 279
Request 1, time: 6.49, n_tokens = 279
Request 9, time: 6.45, n_tokens = 279
Request 7, time: 6.46, n_tokens = 279
Request 4, time: 6.48, n_tokens = 279
Request 2, time: 6.49, n_tokens = 279
Request 8, time: 6.46, n_tokens = 279
Request 5, time: 6.48, n_tokens = 279
Total Execution time: 6.502201795578003


In [17]:
# see response quality
prompt = """
A person is planning a road trip from New York to Los Angeles. 
They have a car with a 15-gallon gas tank, and the fuel efficiency is 25 miles per gallon. 
Assuming they drive for 8 hours a day, how many days will the trip take, and how many times will they need to stop for gas? 
Make assumptions about the route and fuel prices as needed.
"""
getname, response, t = get_response(0, max_tokens=250, system_prompt='You are a helpful assistant.', prompt=prompt)
print(f'Took {t}s')
print(response.choices[0].message.content)

Took 7.888301372528076s
Let's break down the problem step by step!

1. Calculate the daily driving distance:
With 8 hours of driving per day, and a fuel efficiency of 25 miles per gallon, the daily driving distance can be calculated as:

Daily driving distance = Fuel efficiency * 8 hours
= 25 miles/gallon * 8 hours
= 200 miles/day

2. Calculate the total driving distance from New York to Los Angeles:
Assuming a direct route (which is approximately 2,796 miles), we can calculate the total driving distance:

Total driving distance = 2,796 miles

3. Calculate the total fuel consumption:
Since the car has a 15-gallon gas tank, and the fuel efficiency is 25 miles per gallon, we can calculate the total fuel consumption:

Total fuel consumption = Total driving distance / Fuel efficiency
= 2,796 miles / 25 miles/gallon
= 111.84 gallons

4. Calculate the number of days needed for the trip:
Since the car has a 15-gallon gas tank, and it needs to stop for gas when it's empty, we can calculate the